In [0]:
import ast

In [0]:
catalog_raw = dbutils.secrets.get(scope="scope_config_workspace", key="catalog_raw")
catalog_tru = dbutils.secrets.get(scope="scope_config_workspace", key="catalog_trusted")
catalog_ref = dbutils.secrets.get(scope="scope_config_workspace", key="catalog_refined")
catalog_stg = dbutils.secrets.get(scope="scope_config_workspace", key="catalog_stg")

In [0]:
def apply_comments(table: str, schema: str, layer: str, column_comments: dict = None):
    layer_map = {
        "raw"    : catalog_raw,
        "trusted": catalog_tru,
        "refined": catalog_ref,
        "staging": catalog_stg
    }

    if layer not in layer_map:
        raise ValueError(f"Camada inválida: {layer}. Use uma das opções: {list(layer_map.keys())}")

    catalog = layer_map[layer]

    if column_comments:
        if isinstance(column_comments, str):
            column_comments_sql = ast.literal_eval(column_comments)
        sql_commands = []

        for col_name, comment in column_comments_sql.items():
            sql_commands.append(f"""
                ALTER TABLE {catalog}.{schema}.{table}
                ALTER COLUMN `{col_name}`
                COMMENT '{comment}';
            """)
        
        for command in sql_commands:
            spark.sql(command)

In [0]:
table           = dbutils.widgets.get("table")
schema          = dbutils.widgets.get("schema")
layer           = dbutils.widgets.get("layer")
column_comments = dbutils.widgets.get("column_comments")

In [0]:
apply_comments(table, schema, layer, column_comments)